# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** This notebook acts as an index to all the other notebooks

## Table of contents <a name="table-of-contents"></a>

1. [Table of notebooks](#table-notebooks)
2. [Introduction](#introduction)
3. [Collecting the data](#collecting-the-data)
4. [Uploading the data](#uploading-the-data)
5. [Working with the data](#working-with-the-data)
6. [Predicting the future](#predicting-the-future)<br>
    a. [AR models](#AR)<br>
    b. [MA models](#MA)<br>
    c. [ARMA models](#ARMA)<br>
    d. [ARIMA models](#ARIMA)<br>
    e. [Seasonal ARIMA models](#SARIMA)<br>
7. [Conclusion](#conclusion)  
8. [Sources](#sources)

## 1. Table of notebooks <a name="table-notebooks"></a>

A table of all the notebook used in this project can be found here.

1. [Index notebook (current notebook)](./1_entry_notebook.ipynb)
2. [Getting the data](./2_get_data.ipynb)
3. [Some Exploratory Data Analysis (EDA)](./3_exploratory_data_analysis.ipynb)
4. [Selecting our model](./4_model_selection.ipynb)
5. [Fitting the data and predicting the future!](./5_fitting_and_predicting.ipynb)

## 2. Introduction <a name="introduction"></a>

This notebook will give a brief overview of what the assignment was about, how we handled it and a small conclusion.

The goal of the assignment was to predict the temperature values one hour and one day ahead of time. This was done by first collecting temperature data from a chosen location with the [CC3200 LaunchPad](http://www.ti.com/tool/CC3200-LAUNCHXL) development board.


The data the LaunchPad collected was stored in a database hosted on [Heroku](https://heroku.com). An API to upload the data was also hosted on there.

Once enough data is collected it was possible to build these notebooks to accually predict the temperature. This was done by using the [Python](https://www.python.org/) Programming Language in combination with some [modules](./requirements.txt) and [anaconda](https://www.anaconda.com/).

[Jupyter Notebook](http://jupyter.org/) is used which makes it possible to do cool things like [this](./1_entry_notebook.ipynb):

In [2]:
# We <3 Jupyter Notebook
print('Hello, Smart Systems!')

Hello, Smart Systems!


Using Seasonal ARIMAX the future is predicted.

## 3. Collecting the data <a name="collecting-the-data"></a>

As said before, collection of the data is done on the [CC3200 LaunchPad](http://www.ti.com/tool/CC3200-LAUNCHXL) development board. It has an on-board temperature sensor ([TMP006AIYZFR](http://www.ti.com/ww/eu/sensampbook/tmp006.pdf)) that can be used to well... measure temperature!

The temperature is measured and uploaded in intervals of ~15 minutes and uploaded to the server.

The firmware was completely written in [Code Composer Studio](http://www.ti.com/tool/CCSTUDIO) so we could get acces to a lot of usefull included libraries and APIs. TI-RTOS because it has software timers and we were a little to lazy to figure out how we could get the hardware timers to count beyond 5 minutes so we took the easy wasy out. Some improvements could definitely happen in this department. 

It is worth mentioning that the collected temperature values are recorded in degrees Celcius and not Fahrenheit. The location where the data was recorded is also saved.

## 4. Uploading the data <a name="uploading-the-data"></a>

Uploading the actual data to the Heroku server was pretty simple. Using the simplelink API provided in the Code Composer Studio it is simple to:
* Connect to an AP
* Create HTTP requests like POST, PUT and DELETE
* Disconnect from that same AP
* ...

Using this API the board was able to connect to an AP every time it had a new temperature value ready, then create a valid HTTP header and actually send out data toghetter with it. After sending out the data it will disconnect from the AP until it had a new value ready to be uploaded again.

To accually recieve the data an API was created on [Heroku](https://heroku.com) with Python and [Flask](http://flask.pocoo.org/). On there multiple routes were created with Flask to POST, PUT and DELETE data with the development board. On Heroku we also made it possible to send the data to an PostgreSQL database, with the help of some Python modules this was no problem at all.

## 5. Working with the data <a name="working-with-the-data"></a>

After some time passes and the database is filled with data it is to work with that data.

First thing was to "download" the data. This is done by using a Python module named Pandas and SQLAlchemy.
Pandas is used to do 2 things but in this case it was used to act as some sort of web scraper so we can get data directly from the API. SQLAlchemy is used as an alternative to the web scraping technique of pandas. SQLAlchemy directly connects to the database and gets the data via a SQL Querry.

Once the data is "downloaded" a very brief look at the data is taken and a plot is created to give a visual representation.

After this we save the data into a ```.csv``` file this is done because we are dealing with multiple notebooks and the data has to be transferd between them. We could of course reconnect to the database or scrape the web again in every notebook but this takes time. Reading in a ```.csv``` is faster and has the benefit of being able to work offline with them.

The data also comes in the format of a JSON object and everything is a string. This means a conversion has to happen to convert the ```creation_date``` column to a ```datetimeIndex```, and the ```value``` to a ```float```.

When all that is done a closer look at the data is taken. What was concluded here is that not all data is perfect and this one deffinatly has some ugly sides. For example, at a certain point in time the AP the development board was supposed to connect to lost all connection to the internet provider so the development board had nowhere to send the data to. This caused a huge gap to appear in the data.

What can also be observed is that the data is not really steady, it likes to jump. This was fixed by taking the rolling mean of the data.

After taking a closer look and making some pretty graphs it is time to start creating our ARIMA model and accually predicting the future temperatures.

## 6. Predicting the future <a name="predicting-the-future"></a>

Before we are going to predict the futur it might be a good idea to get a general understanding of how we are accually going to do it. Sure we said we are going to use Seasonal ARIMA, but how does this accually work? Well a quick wikipedia search will result in all of the following.

### a. [AR models](https://en.wikipedia.org/wiki/Autoregressive_model) <a name="AR"></a>

In statistics and signal processing, an autoregressive (AR) model is a representation of a type of random process; as such, it is used to describe certain time-varying processes in nature, economics, etc. The autoregressive model specifies that the output variable depends linearly on its own previous values and on a stochastic term (an imperfectly predictable term); thus the model is in the form of a stochastic difference equation.

Together with the moving-average (MA) model, it is a special case and key component of the more general ARMA and ARIMA models of time series, which have a more complicated stochastic structure; it is also a special case of the vector autoregressive model (VAR), which consists of a system of more than one interlocking stochastic difference equation in more than one evolving random variable.

Contrary to the moving-average model, the autoregressive model is not always stationary as it may contain a unit root.

#### Definition

The notation ${\displaystyle AR(p)}$ indicates an autoregressive model of order p. The AR(p) model is defined as
$$X_{t}=c+\sum _{{i=1}}^{p}\varphi _{i}X_{{t-i}}+\varepsilon _{t}$$

where ${\displaystyle \varphi _{1},\ldots ,\varphi _{p}}$ are the parameters of the model, ${\displaystyle c}$ is a constant, and ${\displaystyle \varepsilon _{t}} \varepsilon _{t}$ is white noise. This can be equivalently written using the backshift operator B as

$$X_{t}=c+\sum _{{i=1}}^{p}\varphi _{i}B^{i}X_{t}+\varepsilon _{t}$$

so that, moving the summation term to the left side and using polynomial notation, we have

$${\displaystyle \phi [B]X_{t}=c+\varepsilon _{t}\,.}$$

An autoregressive model can thus be viewed as the output of an all-pole infinite impulse response filter whose input is white noise.

Some parameter constraints are necessary for the model to remain wide-sense stationary. For example, processes in the AR(1) model with ${\displaystyle |\varphi _{1}|\geq 1}$ are not stationary. More generally, for an AR(p) model to be wide-sense stationary, the roots of the polynomial ${\displaystyle \textstyle z^{p}-\sum _{i=1}^{p}\varphi _{i}z^{p-i}}$ must lie inside of the unit circle, i.e., each (complex) root ${\displaystyle z_{i}}$ must satisfy ${\displaystyle |z_{i}|<1}$.

### b. [MA models](https://en.wikipedia.org/wiki/Moving-average_model) <a name="MA"></a>

In time series analysis, the moving-average model (MA model), also known as moving-average process, is a common approach for modeling univariate time series. The moving-average model specifies that the output variable depends linearly on the current and various past values of a stochastic (imperfectly predictable) term.

Together with the autoregressive (AR) model, the moving-average model is a special case and key component of the more general ARMA and ARIMA models of time series, which have a more complicated stochastic structure.

The moving-average model should not be confused with the moving average, a distinct concept despite some similarities.

Contrary to the AR model, the finite MA model is always stationary.

#### Definition

The notation MA(q) refers to the moving average model of order q:

$${\displaystyle X_{t}=\mu +\varepsilon _{t}+\theta _{1}\varepsilon _{t-1}+\cdots +\theta _{q}\varepsilon _{t-q}\,}$$
where μ is the mean of the series, the θ1, ..., θq are the parameters of the model and the εt, εt−1,..., εt−q are white noise error terms. The value of q is called the order of the MA model. This can be equivalently written in terms of the backshift operator B as

$${\displaystyle X_{t}=\mu +(1+\theta _{1}B+\cdots +\theta _{q}B^{q})\varepsilon _{t}.}$$
Thus, a moving-average model is conceptually a linear regression of the current value of the series against current and previous (observed) white noise error terms or random shocks. The random shocks at each point are assumed to be mutually independent and to come from the same distribution, typically a normal distribution, with location at zero and constant scale.

### c. [ARMA models](https://en.wikipedia.org/wiki/Autoregressive%E2%80%93moving-average_model) <a name="ARMA"></a>

In the statistical analysis of time series, autoregressive–moving-average (ARMA) models provide a parsimonious description of a (weakly) stationary stochastic process in terms of two polynomials, one for the autoregression (AR) and the second for the moving average (MA). The general ARMA model was described in the 1951 thesis of Peter Whittle, Hypothesis testing in time series analysis, and it was popularized in the 1970 book by George E. P. Box and Gwilym Jenkins.

Given a time series of data Xt , the ARMA model is a tool for understanding and, perhaps, predicting future values in this series. The AR part involves regressing the variable on its own lagged (i.e., past) values. The MA part involves modeling the error term as a linear combination of error terms occurring contemporaneously and at various times in the past. The model is usually referred to as the ARMA(p,q) model where p is the order of the AR part and q is the order of the MA part (as defined below).

ARMA models can be estimated by using the Box–Jenkins method.

#### Definition

The notation ARMA(p, q) refers to the model with p autoregressive terms and q moving-average terms. This model contains the AR(p) and MA(q) models,

$${\displaystyle X_{t}=c+\varepsilon _{t}+\sum _{i=1}^{p}\varphi _{i}X_{t-i}+\sum _{i=1}^{q}\theta _{i}\varepsilon _{t-i}.\,}$$

The general ARMA model was described in the 1951 thesis of Peter Whittle, who used mathematical analysis (Laurent series and Fourier analysis) and statistical inference. ARMA models were popularized by a 1970 book by George E. P. Box and Jenkins, who expounded an iterative (Box–Jenkins) method for choosing and estimating them. This method was useful for low-order polynomials (of degree three or less).

### d. [ARIMA models](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) <a name="ARIMA"></a>

In statistics and econometrics, and in particular in time series analysis, an autoregressive integrated moving average (ARIMA) model is a generalization of an autoregressive moving average (ARMA) model. Both of these models are fitted to time series data either to better understand the data or to predict future points in the series (forecasting). ARIMA models are applied in some cases where data show evidence of non-stationarity, where an initial differencing step (corresponding to the "integrated" part of the model) can be applied one or more times to eliminate the non-stationarity.[1]

The AR part of ARIMA indicates that the evolving variable of interest is regressed on its own lagged (i.e., prior) values. The MA part indicates that the regression error is actually a linear combination of error terms whose values occurred contemporaneously and at various times in the past. The I (for "integrated") indicates that the data values have been replaced with the difference between their values and the previous values (and this differencing process may have been performed more than once). The purpose of each of these features is to make the model fit the data as well as possible.

Non-seasonal ARIMA models are generally denoted ARIMA(p,d,q) where parameters p, d, and q are non-negative integers, p is the order (number of time lags) of the autoregressive model, d is the degree of differencing (the number of times the data have had past values subtracted), and q is the order of the moving-average model. Seasonal ARIMA models are usually denoted ARIMA(p,d,q)(P,D,Q)m, where m refers to the number of periods in each season, and the uppercase P,D,Q refer to the autoregressive, differencing, and moving average terms for the seasonal part of the ARIMA model.[2][3]

When two out of the three terms are zeros, the model may be referred to based on the non-zero parameter, dropping "AR", "I" or "MA" from the acronym describing the model. For example, ARIMA (1,0,0) is AR(1), ARIMA(0,1,0) is I(1), and ARIMA(0,0,1) is MA(1).

ARIMA models can be estimated following the Box–Jenkins approach.

#### Definition

Given a time series of data Xt where t is an integer index and the Xt are real numbers, an ${\displaystyle ARMA(p',q)}$ model is given by

$${\displaystyle X_{t}-\alpha _{1}X_{t-1}-\dots -\alpha _{p'}X_{t-p'}=\varepsilon _{t}+\theta _{1}\varepsilon _{t-1}+\cdots +\theta _{q}\varepsilon _{t-q},}$$
or equivalently by

$${\displaystyle \left(1-\sum _{i=1}^{p'}\alpha _{i}L^{i}\right)X_{t}=\left(1+\sum _{i=1}^{q}\theta _{i}L^{i}\right)\varepsilon _{t}\,}$$

where ${\displaystyle L}$ is the lag operator, the ${\displaystyle \alpha _{i}}$ are the parameters of the autoregressive part of the model, the ${\displaystyle \theta _{i}}$ are the parameters of the moving average part and the ${\displaystyle \varepsilon _{t}}$$ are error terms. The error terms $${\displaystyle \varepsilon _{t}}$ are generally assumed to be independent, identically distributed variables sampled from a normal distribution with zero mean.

Assume now that the polynomial ${\displaystyle \textstyle \left(1-\sum _{i=1}^{p'}\alpha _{i}L^{i}\right)}$ has a unit root (a factor ${\displaystyle (1-L)}$) of multiplicity d. Then it can be rewritten as:

$${\displaystyle \left(1-\sum _{i=1}^{p'}\alpha _{i}L^{i}\right)=\left(1-\sum _{i=1}^{p'-d}\phi _{i}L^{i}\right)\left(1-L\right)^{d}.}$$
An ARIMA(p,d,q) process expresses this polynomial factorisation property with p=p'−d, and is given by:

$${\displaystyle \left(1-\sum _{i=1}^{p}\phi _{i}L^{i}\right)(1-L)^{d}X_{t}=\left(1+\sum _{i=1}^{q}\theta _{i}L^{i}\right)\varepsilon _{t}\,}$$
and thus can be thought as a particular case of an ARMA(p+d,q) process having the autoregressive polynomial with d unit roots. (For this reason, no ARIMA model with d > 0 is wide sense stationary.)

The above can be generalized as follows.

$${\displaystyle \left(1-\sum _{i=1}^{p}\phi _{i}L^{i}\right)(1-L)^{d}X_{t}=\delta +\left(1+\sum _{i=1}^{q}\theta _{i}L^{i}\right)\varepsilon _{t}.\,}$$
This defines an ARIMA(p,d,q) process with **drift** ${\displaystyle {\frac {\delta }{1-\sum \phi _{i}}}}$.

### e. Seasonal ARIMA models <a name="SARIMA"></a>

And than finally we arive at Seasonal ARIMA the model we are going to try and use.The only real difference betweeon Seasonal ARIMA and ARIMA is that we intoduce 4 more parameteres. (P, D, Q)s.

The s parameter or seasonality makes it so that the mean of the observations is not constant,
but instead evolves according to a cyclical pattern:

* For example, in a series of monthly temperatures in Europe the mean temperature is not constant, since it varies by month, but for the same month in different years we can expect a constant average value.

The seasonal period, s, defines the number of observations that make up a seasonal cycle. For example, s = 12 for monthly series, s = 4 for quarterly series, etc.

So in our case we will have a seasonal period of 24. Because we noticed a seasonal effect every day an we sample our data every 1 hour. And since there are 24 hours in a day we end up with a seasonal period of 24.

All the other parameters of both the ARIMA and SARIMA (seasonal ARIMA) model will be produced by running a grid search.

## 7. Conclusion <a name="conclusion"></a>



## 8. Sources <a name="sources"></a>

* [DigitalOcean - A Guide to Time Series Forecasting with ARIMA in Python 3](https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3)
* [Berhane - CO2 Emission Forecast with Python (Seasonal ARIMA)
](https://www.kaggle.com/berhag/co2-emission-forecast-with-python-seasonal-arima)
* Wikipedia (see the title links)
* Google

**[⬆ back to top](#table-of-contents)**<br>
[next notebook](./2_get_data.ipynb)